In [77]:
import pandas_datareader.data as web
import pandas as pd
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import importlib
import plotly.express as px

#Change me!
start = datetime.datetime.today() + relativedelta.relativedelta(years=-5)
split_unit = "6M"
modulename = "BacktestingStrategies.Strategy_RsiOscillator"
classname = "btst.RsiOscillator_Entry50_WithShortPosition"
end = datetime.date.today()

data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
data = data.astype("double")

initial = 1000_000
fname = f"html/plot-"

In [78]:
#日付データを検証期間単位ごとにSplit
#https://kakakakakku.hatenablog.com/entry/2021/05/24/002542
grouped_data = data.groupby(pd.Grouper(freq = split_unit))
dfs = [group for _, group in grouped_data]

In [79]:
#検証期間単位ごとにバックテスト＆結果を蓄積
#親ディレクトリ経由で呼び出し   # https://qiita.com/yokohama4580/items/466a483ae022d264c8ee
import os
import sys
sys.path.append(os.pardir)

btst = importlib.import_module(modulename)
# from BacktestingStrategies import Strategy_RsiOscillator as btst
importlib.reload(btst)
import warnings
warnings.simplefilter('ignore')

returns = []

for monthly_data in dfs:
    # バックテストを設定
    bt = Backtest(
        monthly_data, # チャートデータ
        eval(classname), # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    returns.append(output)

In [80]:
import statistics

df = pd.DataFrame(returns)

#最低と最大の期間のBacktestingを表示＆実行
lines = "１期間単位: " + split_unit + "\n"
lines += "テスト期間: " + str(len(returns)) + "\n"
res = round(statistics.mean(df["Return [%]"]), 4)

#リターンが最小のものの期間を取得
min_return = df[df["# Trades"] != 0]["Return [%]"].min()
min_period_start = pd.to_datetime(df[df["Return [%]"] == min_return]["Start"]).values[0]
min_period_end = pd.to_datetime(df[df["Return [%]"] == min_return]["End"]).values[0]
df_min = data[min_period_start : min_period_end]

#リターンが2番目、3番目に小さいものの期間を取得
min_return_3 = df[df["# Trades"] != 0]["Return [%]"].nsmallest(3)
min2_period_start = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[1]]["Start"]).values[0]
min2_period_end = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[1]]["End"]).values[0]
min3_period_start = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[2]]["Start"]).values[0]
min3_period_end = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[2]]["End"]).values[0]
df_min2 = data[min2_period_start : min2_period_end]
df_min3 = data[min3_period_start : min3_period_end]

#リターンが最大のものの期間を取得
max_return = df[df["# Trades"] != 0]["Return [%]"].max()
max_period_start = pd.to_datetime(df[df["Return [%]"] == max_return]["Start"]).values[0]
max_period_end = pd.to_datetime(df[df["Return [%]"] == max_return]["End"]).values[0]
df_max = data[max_period_start : max_period_end]

lines += "平均Return [%]: " + str(res) + "\n"
lines += "最大Return [%]: " + str(max_return) + "\n"
lines += "最大Return期間: " + str(max_period_start) + " - " + str(max_period_end) + "\n"
lines += "最低Return [%]: " + str(min_return) + "\n"
lines += "最低Return期間: " + str(min_period_start) + " - " + str(min_period_end) 

print(lines)



１期間単位: 6M
テスト期間: 11
平均Return [%]: -3.9932
最大Return [%]: 18.903805700000003
最大Return期間: 2019-10-01T00:00:00.000000000 - 2020-03-31T00:00:00.000000000
最低Return [%]: -25.961170070000005
最低Return期間: 2018-04-02T00:00:00.000000000 - 2018-09-28T00:00:00.000000000


In [81]:
#取引ない月込み
fig = px.box(df["Return [%]"], points = "all")
fig.update_layout(
    xaxis_title = "Strategy",
    yaxis_title = "Return [%]"
)
fig.show()

In [82]:
#最低と最高の期間のバックテストを表示

print("----------WORST----------")
bt = Backtest(
    df_min, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min.html") # 実行結果（グラフ）

print("----------MIN2----------")
bt = Backtest(
    df_min2, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min2.html") # 実行結果（グラフ）

print("----------MIN3----------")
bt = Backtest(
    df_min3, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min3.html") # 実行結果（グラフ）

print("----------BEST----------")
bt = Backtest(
    df_max, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_max.html") # 実行結果（グラフ）

----------WORST----------
Start                     2018-04-02 00:00:00
End                       2018-09-28 00:00:00
Duration                    179 days 00:00:00
Exposure Time [%]                   36.923077
Equity Final [$]                  740388.2993
Equity Peak [$]                     1000000.0
Return [%]                          -25.96117
Buy & Hold Return [%]              -25.256223
Return (Ann.) [%]                  -44.159107
Volatility (Ann.) [%]                9.337603
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -25.96117
Avg. Drawdown [%]                   -25.96117
Max. Drawdown Duration      122 days 00:00:00
Avg. Drawdown Duration      122 days 00:00:00
# Trades                                    4
Win Rate [%]                              0.0
Best Trade [%]                      -6.208604
Worst Trade [%]                     -8.079405
Avg. Tra

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,774,42,49,1290.3558,1191.0,-76901.3892,-0.076999,2018-05-30,2018-06-08,9 days
1,743,57,77,1242.1182,1165.0,-57298.8226,-0.062086,2018-06-20,2018-07-18,28 days
2,691,95,106,1252.1677,1151.0,-69906.8807,-0.080794,2018-08-13,2018-08-28,15 days
3,653,114,120,1217.9994,1133.0,-55504.6082,-0.069786,2018-09-07,2018-09-17,10 days


----------MIN2----------
Start                     2020-10-02 00:00:00
End                       2021-03-31 00:00:00
Duration                    180 days 00:00:00
Exposure Time [%]                   18.852459
Equity Final [$]                  781317.1089
Equity Peak [$]                     1000000.0
Return [%]                         -21.868289
Buy & Hold Return [%]              -42.243767
Return (Ann.) [%]                  -39.934243
Volatility (Ann.) [%]                9.098588
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -22.809289
Avg. Drawdown [%]                  -22.809289
Max. Drawdown Duration      152 days 00:00:00
Avg. Drawdown Duration      152 days 00:00:00
# Trades                                    4
Win Rate [%]                              0.0
Best Trade [%]                       -1.20673
Worst Trade [%]                     -8.181773
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1393,21,23,717.5343,665.0,-73180.2799,-0.073215,2020-11-02,2020-11-05,3 days
1,1905,81,84,486.3958,452.0,-65523.9990,-0.070716,2021-02-01,2021-02-04,3 days
2,1947,103,111,442.1780,406.0,-70438.5660,-0.081818,2021-03-05,2021-03-17,12 days
3,1882,115,121,420.0691,415.0,-9540.0462,-0.012067,2021-03-23,2021-03-31,8 days


----------MIN3----------
Start                     2022-04-01 00:00:00
End                       2022-09-20 00:00:00
Duration                    172 days 00:00:00
Exposure Time [%]                   47.413793
Equity Final [$]                  857135.7723
Equity Peak [$]                  1018505.5272
Return [%]                         -14.286423
Buy & Hold Return [%]               -5.912596
Return (Ann.) [%]                  -28.458824
Volatility (Ann.) [%]                 17.7164
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.074613
Avg. Drawdown [%]                  -20.074613
Max. Drawdown Duration      123 days 00:00:00
Avg. Drawdown Duration      123 days 00:00:00
# Trades                                    6
Win Rate [%]                        33.333333
Best Trade [%]                       1.850609
Worst Trade [%]                     -10.08364
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2348,30,31,425.88140,418.0,18505.52720,0.018506,2022-05-19,2022-05-20,1 days
1,-2552,32,33,399.01505,401.0,-5065.59240,-0.004975,2022-05-23,2022-05-24,1 days
2,2465,34,42,411.02455,384.0,-66615.51575,-0.065749,2022-05-25,2022-06-06,12 days
3,2270,48,74,417.05425,375.0,-95463.14750,-0.100836,2022-06-14,2022-07-21,37 days
4,2366,101,112,359.77210,363.0,7637.21140,0.008972,2022-08-30,2022-09-14,15 days
5,2367,113,115,362.78695,362.0,-1862.71065,-0.002169,2022-09-15,2022-09-20,5 days


----------BEST----------
Start                     2019-10-01 00:00:00
End                       2020-03-31 00:00:00
Duration                    182 days 00:00:00
Exposure Time [%]                   26.666667
Equity Final [$]                  1189038.057
Equity Peak [$]                   1189038.057
Return [%]                          18.903806
Buy & Hold Return [%]               15.318744
Return (Ann.) [%]                   43.850404
Volatility (Ann.) [%]               34.391672
Sharpe Ratio                          1.27503
Sortino Ratio                        3.465476
Calmar Ratio                         3.473708
Max. Drawdown [%]                  -12.623513
Avg. Drawdown [%]                   -5.764326
Max. Drawdown Duration       50 days 00:00:00
Avg. Drawdown Duration       18 days 00:00:00
# Trades                                    4
Win Rate [%]                             75.0
Best Trade [%]                      14.874961
Worst Trade [%]                     -5.764479
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1098,63,68,910.48470,858.0,-57628.20060,-0.057645,2020-01-07,2020-01-15,8 days
1,1027,77,98,917.51935,1054.0,140165.62755,0.148750,2020-01-28,2020-02-28,31 days
2,-878,116,117,1232.86695,1179.0,47295.18210,0.043692,2020-03-26,2020-03-27,1 days
3,-913,118,119,1236.84715,1172.0,59205.44795,0.052429,2020-03-30,2020-03-31,1 days


Row(id='44564', ...)